In [1]:
import pandas
import datetime
import time
# import the dataset creation module
import os
import sys
module_path = os.path.abspath(os.path.join('../data_pipeline/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import create_onset_data
import omn_utils
import batch_utils

/usr/local/lib/python2.7/dist-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/dist-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/dist-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/usr/local/lib/python2.7/dist-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/usr/local/lib/python2.7/dist-packages/pandas/core/indexes/base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate bi

In [2]:
# dataObj = create_onset_data.OnsetData()

In [3]:
# print len(dataObj.polarDateList), len(dataObj.imageDateList)

In [4]:
# x = time.time()
# finDF = dataObj.create_output_bins()
# y = time.time()
# print y-x

In [5]:
# finDF.head()

In [6]:
# finDF = pandas.read_feather("../data/binned_data.feather")
# finDF.head()

In [7]:
omn_dbdir = "/home/bharat/Documents/data/ss_onset_dataset/data/sqlite3/"
omn_db_name = "omni_sw_imf.sqlite"
omn_table_name = "imf_sw"
omn_train = True
omn_norm_param_file = omn_dbdir + "omn_mean_std.npy"
# start_date = finDF["date"].min()
# end_date = finDF["date"].max()
# print start_date, end_date

In [8]:
# omnObj = omn_utils.OmnData(start_date, end_date, omn_dbdir,\
#                            omn_db_name, omn_table_name,\
#                            omn_train, omn_norm_param_file)
# omnObj.omnDF.head()

In [9]:
batchObj = batch_utils.DataUtils(omn_dbdir,\
                           omn_db_name, omn_table_name,\
                           omn_train, omn_norm_param_file,\
                                onsetDataloadFile="../data/binned_data_extra_latlon.feather")

loading precomputed onset data
normalizing the IMF data ...
mean and std values...
(By      0.213039
Bz     -0.083170
Bx     -0.166686
Vx   -430.806568
Np      6.842641
dtype: float64, By     4.696615
Bz     3.948871
Bx     4.116263
Vx    95.882814
Np     5.498686
dtype: float64)


In [10]:
x = time.time()
b1 = batchObj.batchDict[0]
y = time.time()
outOn = batchObj.onset_from_batch(b1)
z = time.time()
inOmn = batchObj.omn_from_batch(b1)
w = time.time()

print "batch calc--->", z-y
print "outOn calc--->", z-y
print "inOmn calc--->", w-z

batch calc---> 0.0723340511322
outOn calc---> 0.0723340511322
inOmn calc---> 0.0882208347321


/home/bharat/Documents/code/sson_pred/data_pipeline/batch_utils.py:130: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  self.onsetDF.index.isin(dateList)\
/home/bharat/Documents/code/sson_pred/data_pipeline/batch_utils.py:150: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  self.omnDF.loc[ _et : _st ].as_matrix())


In [11]:
print outOn.shape
print inOmn.shape

(64, 1, 10)
(64, 121, 5)
